In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

data=pd.read_csv(r"C:\Users\19692\Downloads\processed_data_mileage.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8611 entries, 0 to 8610
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           8611 non-null   int64  
 1   year                                 8611 non-null   int64  
 2   make                                 8611 non-null   object 
 3   model                                8611 non-null   object 
 4   price                                8611 non-null   int64  
 5   mileage                              8611 non-null   int64  
 6   Miles per gallon                     8611 non-null   object 
 7   Transmission                         8594 non-null   object 
 8   owner                                8611 non-null   float64
 9   VIN                                  8611 non-null   object 
 10  class                                8611 non-null   object 
 11  Auction Brand / Issues        

In [86]:
# add correlation and price_drop/mile for each make
brand_corr = data.groupby('make')
brand_corr=brand_corr.apply(lambda x: x['mileage'].corr(x['price']))
brand_corr

def brand_slope(brand_data):
    X = X = brand_data['mileage'].values.reshape(-1, 1) 
    y = brand_data['price']
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_[0]
brand_slopes = data.groupby('make').apply(brand_slope)

data['brand_corr']=data['make'].map(brand_corr)
data['price_drop_per_mile']=data['make'].map(brand_slopes)

C:\Users\19692\AppData\Local\Temp\ipykernel_14260\559903387.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  brand_corr=brand_corr.apply(lambda x: x['mileage'].corr(x['price']))
C:\Users\19692\AppData\Local\Temp\ipykernel_14260\559903387.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  brand_slopes = data.groupby('make').apply(brand_slope)


In [87]:
data.dropna(subset=['year','price','mileage','accident_encoded','fuel_encoded','owner','brand_corr','price_drop_per_mile',])

,Unnamed: 0,year,make,model,price,mileage,Miles per gallon,Transmission,owner,VIN,...,Bed Length,exterior_color,interior_color,city_mpg,hwy_mpg,accident_encoded,fuel_encoded,drive_type_encoded,brand_corr,price_drop_per_mile
0,1,2017,Subaru,Forester 2.5I,16998,79000,22 city/28 hwy,Manual 6 Speed,1.0,JF2SJAACXHG452398,...,NaN,Silver,Gray,22,28,3,1,0,-0.801206,-0.152142
1,2,2020,Subaru,Impreza,18998,43000,28 city/36 hwy,Automatic,1.0,4S3GKAB64L3608865,...,NaN,Blue,Black,28,36,5,1,0,-0.801206,-0.152142
2,3,2016,Subaru,Legacy 2.5I,14998,91000,26 city/36 hwy,Automatic,3.0,4S3BNAA63G3004121,...,NaN,Gray,Black,26,36,5,1,0,-0.801206,-0.152142
3,4,2021,Subaru,Crosstrek Premium,24998,53000,28 city/33 hwy,Automatic,1.0,JF2GTAEC8MH670080,...,NaN,Black,Black,28,33,5,1,0,-0.801206,-0.152142
4,5,2017,Subaru,Crosstrek Premium,16998,102000,26 city/33 hwy,Automatic,2.0,JF2GPABC9H8207377,...,NaN,Black,Black,26,33,5,1,0,-0.801206,-0.152142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8606,10358,2021,Acura,RDX A-Spec,36998,10000,22 city/27 hwy,Automatic,1.0,5J8TC1H66ML023257,...,NaN,White,Red,22,27,5,6,2,-0.815135,-0.235028
8607,10359,2022,Acura,MDX SH-AWD,41998,12000,19 city/25 hwy,Automatic,1.0,5J8YE1H30NL042278,...,NaN,White,Black,19,25,5,1,0,-0.815135,-0.235028
8608,10361,2016,Acura,MDX,18998,89000,19 city/27 hwy,Automatic,2.0,5FRYD3H21GB014398,...,NaN,Gray,Black,19,27,3,1,2,-0.815135,-0.235028
8609,10362,2020,Acura,MDX Technology,22998,117000,20 city/27 hwy,Automatic,1.0,5J8YD3H59LL010345,...,NaN,Blue,Gray,20,27,5,1,2,-0.815135,-0.235028


In [88]:
data.dropna(subset=['year','price','mileage','accident_encoded','fuel_encoded','owner','brand_corr','price_drop_per_mile'])

,Unnamed: 0,year,make,model,price,mileage,Miles per gallon,Transmission,owner,VIN,...,Bed Length,exterior_color,interior_color,city_mpg,hwy_mpg,accident_encoded,fuel_encoded,drive_type_encoded,brand_corr,price_drop_per_mile
0,1,2017,Subaru,Forester 2.5I,16998,79000,22 city/28 hwy,Manual 6 Speed,1.0,JF2SJAACXHG452398,...,NaN,Silver,Gray,22,28,3,1,0,-0.801206,-0.152142
1,2,2020,Subaru,Impreza,18998,43000,28 city/36 hwy,Automatic,1.0,4S3GKAB64L3608865,...,NaN,Blue,Black,28,36,5,1,0,-0.801206,-0.152142
2,3,2016,Subaru,Legacy 2.5I,14998,91000,26 city/36 hwy,Automatic,3.0,4S3BNAA63G3004121,...,NaN,Gray,Black,26,36,5,1,0,-0.801206,-0.152142
3,4,2021,Subaru,Crosstrek Premium,24998,53000,28 city/33 hwy,Automatic,1.0,JF2GTAEC8MH670080,...,NaN,Black,Black,28,33,5,1,0,-0.801206,-0.152142
4,5,2017,Subaru,Crosstrek Premium,16998,102000,26 city/33 hwy,Automatic,2.0,JF2GPABC9H8207377,...,NaN,Black,Black,26,33,5,1,0,-0.801206,-0.152142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8606,10358,2021,Acura,RDX A-Spec,36998,10000,22 city/27 hwy,Automatic,1.0,5J8TC1H66ML023257,...,NaN,White,Red,22,27,5,6,2,-0.815135,-0.235028
8607,10359,2022,Acura,MDX SH-AWD,41998,12000,19 city/25 hwy,Automatic,1.0,5J8YE1H30NL042278,...,NaN,White,Black,19,25,5,1,0,-0.815135,-0.235028
8608,10361,2016,Acura,MDX,18998,89000,19 city/27 hwy,Automatic,2.0,5FRYD3H21GB014398,...,NaN,Gray,Black,19,27,3,1,2,-0.815135,-0.235028
8609,10362,2020,Acura,MDX Technology,22998,117000,20 city/27 hwy,Automatic,1.0,5J8YD3H59LL010345,...,NaN,Blue,Gray,20,27,5,1,2,-0.815135,-0.235028


In [89]:
X=data[['year','price','accident_encoded','fuel_encoded','owner','brand_corr','price_drop_per_mile']]

bins = [0, 20000, 50000, 80000, 110000, 130000]
labels = ['0-20k', '20k-50k', '50k-80k', '80k-110k', '110k-130k'] 
data['mileage_category'] = pd.cut(data['mileage'], bins=bins, labels=labels)

label_encoder=LabelEncoder()
data['mileage_category_encode']=label_encoder.fit_transform(data['mileage_category'])
mileage_category_mapping=dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
reverse_mileage_category_mapping={v:k for k,v in mileage_category_mapping.items()}



In [91]:
y=data['mileage_category_encode'].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [92]:
model = GradientBoostingClassifier(random_state=42)

In [94]:
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search= GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy', verbose=2)
grid_search.fit(X_train,y_train.ravel())

best_parameters = grid_search.best_params_
best_gbr=grid_search.best_estimator_

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


KeyboardInterrupt: 

In [ ]:
y_pred = best_gbr.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))